# Smart Logistic System in Walmart 

### Note:  

1. This is a group-based project, you are only allowed to discuss with your team members.

2. You are not allowed to use any packages to read or write csv files.

In [52]:
import numpy as np   #Numpy package would be the only package you could use for this project.

## Problem1: The Estimated Value for Each Warehouse
#### Note that you are not allowed to use dynamic programming in this problem!

In this problem 1, calculating the estimated total value and accumulated weight of each of 300 warehouses will be helpful for the later decision of cross-warehouse-transshipment solution in problem 2. 

### Step 1: Estimate the Maximized Total Value and Accumulated Weight of One Warehouse

Read the dataset from  "Products.csv" first.

In [53]:
file_name='Products.csv'
#write your codes here
    
f = open(file_name,"r")

lines = f.readlines()

lines = [line.strip("\n").split(",") for line in open(file_name,"r")]


Now let's get the estimated value and weight for one warehouse.You need to write a function "get_max_value" which could do the following 4 things:
1. Calculate the Value_Weight Ratio for each product. 
2. Sort the calculated Value_Weight Ratios in descending order. 
3. Select the products from the top of the descending order to the bottom while looking at the accumulated         weight of products to make sure the accumulated weight does not exceed the warehouse weight capacity.
4. Calculate the  corresponding estimated value for this warehouse after finishing product selection.

Remember, take every product into consideration; otherwise, you may have the risk of missing products. You want to put as many product with highest Value_Weight Ratio as possible before reaching the weight capacity for this warehouse.

In [54]:

def get_max_value(i):          #you should add input variables yourself in the parentheses
    total_value = 0
    total_weight = 0
    remain_capacity = 650
    weight_array = np.array(lines[i]).astype('float') #convert from list to array; convert numbers from str to floating number
    value_array = np.array(lines[i+1]).astype('float')
    
    #calculate the value-weight ratio
    ratio =  value_array / weight_array
    
    #sort the ratio in descending order and get the original index(aka the NO. of product)
    sorted_index = np.flip(np.argsort(ratio))
    
    for i in range(sorted_index.shape[0]):  #sorted_index.shape[0] = the number of products in one warehouse = 50
        index = sorted_index[i]   #get the NO. of product to further check its corresponding value and weight
        if remain_capacity >= weight_array[index]: #check whether there is still capacity for this product
            total_value += value_array[index]   #accumulate the value of this product to the total value of the warehouse
            total_weight += weight_array[index] #accumulate the weight of this product to the total weight of the warehouse
            remain_capacity -= weight_array[index] #renew the remain capacity
        elif remain_capacity == 0: # if there is no capacity in the warehouse, leave the loop
            break
    return total_value, total_weight


    #Make sure you have calculated the total Value and accumulated Weight for One Warehouse

### Step 2: Estimate the Total Value for 300 Warehouses
Let's apply the same calculation process of step 1 for one warehouse to the 299 warehouses left. 

Keep in mind that calculating the estimated total value and accumulated weight of 300 warehouses will be helpful for the later decision of cross-warehouse-transshipment solution in problem 2. 

In [55]:
#write your codes here
warehouse_value = []

''' we have 300 instances and one warehouse has 3 rows on the dataset. So, we will iterate for 900'''
for i in range(0,900,3):
        
    #write your codes here  

    warehouse_value.append(get_max_value(i))

warehouse_value

print(f"{'Warehouse#':<15}{'Total Value':<15}{'Total Weight':<15}")

for i in range(len(warehouse_value)):
    print(f"{i:<15}{warehouse_value[i][0]:<15.2f}{warehouse_value[i][1]:<15}")


Warehouse#     Total Value    Total Weight   
0              1076.27        650.0          
1              976.78         648.0          
2              1231.70        644.0          
3              1003.49        649.0          
4              1099.97        645.0          
5              1009.18        650.0          
6              1196.45        648.0          
7              975.12         647.0          
8              1009.39        648.0          
9              1054.00        649.0          
10             948.03         646.0          
11             1102.16        648.0          
12             1133.12        648.0          
13             1110.68        645.0          
14             938.15         644.0          
15             1059.30        634.0          
16             1174.39        650.0          
17             997.17         649.0          
18             998.07         641.0          
19             986.91         642.0          
20             1221.07        650.

## Problem 2: Top Alternative Selections

In this problem, you will simulate the cross-warehouse-transshipment solutions when a random warehouse is out-of-stock during the promotion period.

### Step 1: Generate the 300*300 distance matrix and write the results to "Distances.csv"
Let's generate a distance matrix among the 300 warehouses first. Each of the distances can be generated by using a normal distribution with a mean of 500 and a standard deviation of 300. 

Be careful, all the distances generated should be positive numbers and should be rounded to integers using the round() function. After successfully generating the distance matrix, please write it to a new CSV file called “Distances.csv”.

In [62]:
# Generate the Distance matrix 300*300

#write your codes here
    
    
distances = np.random.normal(loc= 500, scale=300, size=(300,300))

for r in range(len(distances)):
    for c in range(len(distances[0])):
        if r == c: #renew the distance between itself to zero 
            distances[r][c] = 0
        elif distances[r][c] != distances[c][r]: #make the distance between two warehouse the same
            distances[c][r] = distances[r][c]

distances = np.abs(distances).round() 
    

distances            #show the distance matrix you generated 
  

# Check your maxtrix and see if all numbers are positive and rounded to integer

array([[  0., 477., 378., ..., 502.,  69., 657.],
       [477.,   0., 787., ...,  54., 396., 725.],
       [378., 787.,   0., ..., 592., 170., 583.],
       ...,
       [502.,  54., 592., ...,   0., 882., 548.],
       [ 69., 396., 170., ..., 882.,   0., 255.],
       [657., 725., 583., ..., 548., 255.,   0.]])

Now we are going to save the file in 'Distance.csv'

In [63]:
f = open("Distances.csv", "w") # We will create this file.


#write your codes here
    
    
np.savetxt(f, distances, delimiter=",", fmt = "%d")

    
    
f.close()

### Step2：Select the Alternative Warehouse ("Helper")

Marlon randomly selects No.p warehouse that is out-of-stock in this simulation.

In [64]:
'''p is offered by Malron. Do not change the code below.'''

p=np.random.randint(0,300) 
p

70

There are 3 things to do:
1. Based on the generated p, find the corresponding distance between this warehouse No.p and each of the other Helpers from the distance matrix you generated in Step 1 of Problem 2.
2. Find the corresponding total value and total weight of all products stored in each Helper (you have calculated these numbers in Problem 1).
3. Calculate the “Value_per_Weight” ratio for each Helper, sort the ratio in descending order and choose top helpers with 10 highest “Value_per_Weight”ratios.

###  $Value\_per\_Weight = \frac{Total\_Value}{Total\_Weight}-Distance\times\ Transpotation\_Cost$

#### Note that you are not allowed to use dynamic programming in this problem!

In [65]:
#write your codes here

output = []

p_dis = distances[p] #get the array of the distances to the helpers

vpw_list = []
for i in range(300):
    value = warehouse_value[i][0]
    weight = warehouse_value[i][1]
    dis = p_dis[i]
    vpw = value / weight - dis * 0.015   #calculate the "value_per_weight" rate
    vpw_list += [vpw]

sorted_vpw = np.flip(np.argsort(np.array(vpw_list))) #convert the 'vpw_list' into array, sort and get the original index(aka the NO. of warehouse), change it to descending order 

print(f"{'Rank':<10}{'Index':<10}{'Value_Per_Weight':<10}")

last_vpw = 0
for i in range(sorted_vpw.shape[0]):
        
    if i > 9:    #check whether the 11th helper has the same vpw value with the 10th helper
        if vpw != last_vpw:
            break
            
    if i == 0:  #record the last vpw value to identify the warehouses with the same vpw value at the end of ranking (10th, 11th)
        l_vpw = 0
    else:
        l_vpw = vpw
    
    rank = i + 1    
    index = sorted_vpw[i]  
    vpw = vpw_list[index]
    print(f"{rank:<10}{index:<10}{vpw:<10.2f}")


    

# Show the cross-warehouse-transshipment solution when No.p warehouse is out-of-stock during the promotion period. 
# The solution should contain Helper indexes and the corresponding "Value_Per_Weight" ratios. 

Rank      Index     Value_Per_Weight
1         70        1.75      
2         20        1.70      
3         37        1.16      
4         9         0.93      
5         292       0.88      
6         208       0.72      
7         172       0.59      
8         236       0.39      
9         93        0.24      
10        107       0.12      


#### Note: Due to random assignment of No.p warehouse, it's acceptable if your recommended solution changes each time you re-run the code. 

### Submision Requirement: Please submit all files (Distances.csv, and Fall21_Mid_Walmart.ipynb files) as one zip file on Canvas. Please add SIDs and first names of all teammates on the name of zip file. For example, Zhack_14325_Allen_34672_Alex_12345.zip.